To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + support us if you can!
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

See on our [blog post](https://unsloth.ai/blog/gemma) on how we made **Gemma 7b 2.5x faster** and **Gemma 2x faster**!

In [3]:
# %%capture
import torch
import os

os.environ["WANDB_PROJECT"] = "gemma_2b_test"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
# major_version, minor_version = torch.cuda.get_device_capability()
# if major_version >= 8:
#     # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
#     !pip install "unsloth[colab-ampere] @ git+https://github.com/unslothai/unsloth.git"
# else:
#     # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
#     !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"
# pass

* We support Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes etc
* And Yi, Qwen ([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, all Llama, Mistral derived archs.
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.647 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.22.post7. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.3 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# load json dataset at input.json
dataset = load_dataset('json', data_files='input-small.json', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 43 examples [00:00, 14509.66 examples/s]
Map: 100%|██████████| 43/43 [00:00<00:00, 14504.99 examples/s]


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 1,
        # max_steps = ,
        num_train_epochs = 100,
        learning_rate = 8e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="wandb",
    ),
)

/home/yashas/miniconda3/envs/pytorch_test/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/yashas/miniconda3/envs/pytorch_test/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Map (num_proc=2): 100%|██████████| 43/43 [00:00<00:00, 46.57 examples/s]


In [8]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.647 GB.
2.219 GB of memory reserved.


In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 43 | Num Epochs = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 19,611,648
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yashas-salankimatt (yashas-personal). Use `wandb login --relogin` to force relogin


  0%|          | 1/1000 [00:03<56:42,  3.41s/it]

{'loss': 0.6248, 'grad_norm': 0.2204626500606537, 'learning_rate': 0.002, 'epoch': 0.09}


  0%|          | 2/1000 [00:06<52:19,  3.15s/it]

{'loss': 0.6299, 'grad_norm': 0.11693359166383743, 'learning_rate': 0.001997997997997998, 'epoch': 0.19}


...

100%|██████████| 1000/1000 [46:23<00:00,  2.63s/it]Checkpoint destination directory outputs/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


{'loss': 0.0009, 'grad_norm': 0.005194799043238163, 'learning_rate': 0.0, 'epoch': 93.02}


wandb: Adding directory to artifact (./outputs/checkpoint-1000)... Done. 0.1s
100%|██████████| 1000/1000 [46:24<00:00,  2.63s/it]

{'train_runtime': 2788.1459, 'train_samples_per_second': 1.542, 'train_steps_per_second': 0.359, 'train_loss': 0.028386561334773432, 'epoch': 93.02}


100%|██████████| 1000/1000 [46:25<00:00,  2.79s/it]


In [10]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2788.1459 seconds used for training.
46.47 minutes used for training.
Peak reserved memory = 21.664 GB.
Peak reserved memory for training = 19.445 GB.
Peak reserved memory % of max memory = 91.614 %.
Peak reserved memory for training % of max memory = 82.23 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [11]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "Continue the fibonnaci sequence.", # instruction
#         "1, 1, 2, 3, 5, 8", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

inputs = tokenizer(
[
    alpaca_prompt.format(
    "Given the following few token representations of a visual frame in the following format 'frame t-0: [{{object_id, {{centroid_x, centroid_y}},num_radial_points,radial_points}}]', predict the next two frames in the following format 'frame t+0: [{{object_id, {{centroid_x, centroid_y}},num_radial_points,radial_points}}]' and 'frame t+1: [{{object_id, {{centroid_x, centroid_y}},num_radial_points,radial_points",
    "frame t-3: [{gripper,{231,48},91,92,80,81,85,91,94,96,98,15,14,13,13,13,13,70,82,81,78,74,71,70,94,95,90,87,86,86,88,97,84,72,64,59,55,52,50,50,50,50,52,55,59,64,72,79,91,95,87,86}, {table,{311,309},329,332,340,354,351,288,247,220,201,187,178,173,170,170,173,178,187,201,220,247,288,349,336,323,315,312,315,323,336,317,298,284,277,267,224,229,181,195,193,190,190,192,198,208,224,245,276,320,340,332}, {yellow block,{210,313},43,40,36,33,32,31,30,30,30,31,33,35,39,44,50,51,50,49,50,50,51,50,49,48,47,43,37,33,30,29,28,27,26,27,27,29,30,33,37,41,48,49,49,50,52,54,53,50,49,48}, {green block,{166,217},42,43,38,33,31,29,27,26,27,26,27,27,30,31,34,38,42,42,40,40,41,42,41,41,41,43,43,37,33,30,26,27,25,24,24,25,25,26,28,30,34,38,40,41,42,43,44,42,42,43}, {blue block,{387,328},45,44,40,36,34,33,31,32,31,31,33,35,38,42,47,52,50,49,49,49,50,51,48,46,45,45,42,37,35,32,30,30,30,29,29,30,32,34,38,42,48,50,49,49,50,52,54,52,49,47}, {red block,{345,196},39,40,41,42,42,40,40,40,41,37,32,29,28,27,25,25,25,25,25,27,28,31,35,39,39,39,40,41,41,40,38,38,38,37,34,29,27,25,24,23,23,23,23,24,25,28,30,33,39,40}]\nframe t-2: [{gripper,{231,48},92,93,81,82,85,91,95,96,99,16,14,14,14,14,14,71,82,81,77,74,71,69,94,94,89,86,85,85,88,96,84,72,64,59,55,52,50,50,50,50,52,55,59,64,72,77,90,95,88,86}, {table,{311,309},329,332,340,354,351,288,247,220,201,187,179,173,170,170,173,178,187,201,220,247,288,349,336,323,315,312,315,323,336,316,297,284,276,265,222,228,180,193,192,190,190,192,198,208,223,244,274,318,340,332}, {yellow block,{211,314},42,38,34,33,31,29,29,29,29,31,32,34,37,42,48,50,49,49,49,50,51,50,49,48,47,44,38,34,30,28,28,27,27,27,28,29,30,33,37,43,48,49,49,50,52,54,53,50,49,47}, {green block,{166,218},42,42,37,33,30,29,28,26,27,26,27,28,30,32,35,38,41,41,39,39,40,40,40,40,41,42,44,37,32,28,26,25,25,24,24,25,25,26,27,30,32,39,41,41,42,43,44,44,43,43}, {blue block,{388,328},45,43,39,36,34,33,31,30,30,31,33,35,38,41,47,51,50,49,49,50,50,50,49,47,45,45,43,37,34,32,31,30,30,29,29,30,32,34,38,42,47,49,49,49,50,52,52,51,48,46}, {red block,{345,197},40,41,41,42,42,41,40,40,38,35,32,29,27,26,24,24,24,24,26,27,28,30,34,37,38,39,40,40,41,40,38,38,38,37,35,30,28,26,24,23,23,24,24,25,25,28,30,35,40,41}]\nframe t-1: [{gripper,{231,54},89,93,90,83,86,91,98,98,102,18,17,17,17,17,17,76,84,83,78,75,71,69,68,95,91,88,86,87,89,92,94,81,72,66,61,58,56,56,56,56,58,61,66,70,69,74,83,93,89,87}, {table,{311,309},329,332,340,354,351,290,249,221,201,188,179,174,171,171,174,179,188,202,221,249,290,349,336,323,315,312,315,323,336,316,297,284,277,211,219,230,171,187,193,191,190,193,198,209,224,245,276,320,340,332}, {yellow block,{211,314},42,38,35,33,31,29,29,29,29,30,32,34,37,42,48,51,50,49,49,50,51,50,49,47,47,44,37,33,30,28,28,27,27,27,27,29,30,33,38,43,48,49,49,50,52,54,53,50,48,47}, {green block,{166,218},43,43,37,33,30,29,28,27,27,27,27,28,30,33,35,38,41,41,40,40,41,42,40,41,41,43,44,37,32,27,26,25,25,24,24,25,25,26,27,29,31,38,42,42,42,43,44,44,43,43}, {blue block,{388,328},45,44,39,36,34,31,31,30,30,31,33,35,38,41,48,52,50,49,49,50,50,50,49,47,46,45,43,37,34,32,31,30,29,29,29,30,31,34,37,41,47,49,49,49,50,52,53,51,48,46}, {red block,{345,197},40,41,42,42,43,41,41,41,40,36,32,29,27,26,25,24,24,24,26,27,28,30,34,37,38,39,40,41,41,40,38,38,38,37,35,30,28,26,24,23,23,24,24,25,25,28,30,35,41,41}]\nframe t-0: [{gripper,{233,64},87,89,95,94,90,94,102,106,109,107,30,29,29,29,80,92,90,82,76,72,70,67,66,69,95,92,89,90,92,94,97,95,85,77,72,69,67,66,66,67,69,72,77,77,68,62,63,71,86,91}, {table,{312,311},328,331,339,353,349,286,246,219,199,186,177,172,169,169,172,177,186,199,219,246,286,347,337,324,316,313,316,324,337,319,300,286,278,191,210,230,152,167,196,193,192,195,201,211,227,247,279,324,339,331}, {yellow block,{211,314},42,38,35,33,31,29,29,29,29,30,32,34,37,42,48,51,50,49,50,50,51,50,48,47,47,44,38,33,30,28,28,27,27,27,27,29,30,33,38,44,48,49,49,50,52,54,53,50,48,47}, {green block,{166,218},43,42,37,33,30,29,28,27,27,27,27,28,31,33,35,38,41,41,40,40,41,42,40,41,41,43,44,37,33,27,26,25,25,24,24,25,25,26,27,29,31,38,42,42,42,43,44,44,43,43}, {blue block,{388,328},45,44,39,36,34,31,31,30,30,31,33,35,38,42,47,51,51,50,49,50,50,51,49,47,46,45,43,37,34,32,31,30,29,28,29,30,31,33,37,40,47,49,49,49,50,52,53,51,48,46}, {red block,{345,197},40,41,42,42,43,41,41,40,40,37,32,29,28,26,25,24,24,24,25,27,28,30,34,37,38,39,40,41,41,40,38,38,38,37,35,31,28,26,25,24,23,24,24,25,25,28,30,36,41,41}]",
    "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
tokenizer.batch_decode(outputs)

["<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven the following few token representations of a visual frame in the following format 'frame t-0: [{{object_id, {{centroid_x, centroid_y}},num_radial_points,radial_points}}]', predict the next two frames in the following format 'frame t+0: [{{object_id, {{centroid_x, centroid_y}},num_radial_points,radial_points}}]' and 'frame t+1: [{{object_id, {{centroid_x, centroid_y}},num_radial_points,radial_points\n\n### Input:\nframe t-3: [{gripper,{231,48},91,92,80,81,85,91,94,96,98,15,14,13,13,13,13,70,82,81,78,74,71,70,94,95,90,87,86,86,88,97,84,72,64,59,55,52,50,50,50,50,52,55,59,64,72,79,91,95,87,86}, {table,{311,309},329,332,340,354,351,288,247,220,201,187,178,173,170,170,173,178,187,201,220,247,288,349,336,323,315,312,315,323,336,317,298,284,277,267,224,229,181,195,193,190,190,192,198,208,224,245,276,320,

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [21]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "Continue the fibonnaci sequence for the next 6 numbers.", # instruction
#         "1, 1, 2, 3, 5, 8, 13, 21", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

inputs = tokenizer(
[
    alpaca_prompt.format(
    "Given the following few token representations of a visual frame in the following format 'frame t-0: [{{object_id, {{centroid_x, centroid_y}},num_radial_points,radial_points}}]'..., predict the next frame in the same format'",
    "frame t-2: [{gripper,{172,132},78,70,73,81,86,93,105,158,187,186,202,141,156,156,116,26,22,18,16,16,14,14,14,14,15,16,16,179,120,109,167,195,173,158,147,140,136,134,134,123,113,103,95,84,77,74,72,73,77,77}, {table,{320,318},320,323,331,345,335,274,236,209,191,178,170,164,162,162,164,170,178,191,209,236,274,335,346,332,324,321,324,332,346,331,312,298,291,133,130,237,219,208,201,197,197,199,205,217,231,254,285,332,331,323}, {yellow block,{191,315},26,28,28,32,34,35,36,37,38,39,43,41,40,39,37,36,36,36,36,35,34,32,32,31,32,33,32,29,28,27,26,27,27,27,29,30,32,35,41,47,49,48,42,37,33,30,28,27,26,27}, {green block,{149,219},25,27,27,28,29,19,14,11,8,6,6,6,37,37,34,32,30,29,28,27,25,24,24,24,25,26,27,27,25,21,20,19,19,17,17,17,18,19,20,21,25,28,34,35,32,29,28,26,26,26}, {blue block,{387,328},46,44,40,36,34,33,31,30,31,33,33,35,39,43,48,51,50,49,49,49,50,50,48,47,46,45,41,37,33,31,29,28,28,28,28,29,30,32,36,40,47,50,49,49,50,52,54,53,50,48}, {red block,{345,197},40,41,42,42,43,41,41,41,40,37,32,30,28,27,25,24,24,25,25,27,28,30,34,37,39,39,40,41,41,40,38,38,38,37,34,31,28,26,25,24,24,24,24,25,27,28,30,36,41,41}]\nframe t-1: [{gripper,{177,121},61,63,65,67,76,82,127,151,152,168,178,116,127,140,118,101,87,81,30,31,30,29,28,27,25,24,25,25,191,144,138,179,159,145,135,129,125,123,123,125,107,92,83,78,78,72,69,58,58,59}, {table,{318,319},322,325,333,347,333,273,234,208,190,177,169,163,161,161,163,169,178,190,208,234,273,333,344,330,322,319,322,330,344,330,312,297,290,86,174,238,221,210,203,199,198,201,208,219,234,256,287,334,333,325}, {yellow block,{191,278},25,27,27,31,35,35,36,37,38,40,43,40,39,38,37,36,36,36,37,35,34,32,33,33,33,34,32,29,28,27,26,27,27,28,29,30,33,37,42,45,47,45,38,33,31,29,27,25,25,26}, {green block,{140,230},14,15,16,18,20,23,28,30,30,29,28,25,22,20,17,16,15,15,13,13,13,13,12,13,13,14,14,15,16,18,22,24,25,25,24,25,26,27,18,15,14,15,15,15,15,15,15,14,14,15}, {blue block,{387,328},45,44,40,36,34,33,31,30,31,33,33,35,39,43,47,51,50,49,49,49,50,50,48,47,46,45,41,37,33,32,29,28,28,28,28,29,30,32,36,40,47,49,49,49,50,52,54,53,50,48}, {red block,{345,197},40,41,42,42,43,41,41,40,40,37,32,30,28,27,25,24,24,25,26,27,29,30,34,37,38,39,40,41,41,40,38,38,38,37,34,31,28,27,25,24,24,24,24,25,27,28,31,36,41,41}]\nframe t-0: [{gripper,{180,118},46,49,50,52,55,94,103,124,116,137,147,87,87,101,117,104,90,87,85,75,74,36,35,36,36,38,40,46,50,53,56,70,155,141,132,126,122,120,120,105,93,79,58,54,53,49,46,46,45,44}, {table,{317,319},323,326,334,348,333,273,234,208,190,177,169,163,161,161,163,169,177,190,208,234,273,333,343,329,321,318,321,329,343,329,311,297,120,125,212,233,221,210,203,199,199,202,208,219,234,256,288,335,334,326}, {yellow block,{183,247},26,28,29,34,36,36,36,37,40,41,40,38,38,37,36,37,38,37,36,35,33,32,32,33,33,33,32,29,27,27,26,27,27,28,29,31,33,39,44,46,46,40,36,32,31,29,27,26,25,26}, {green block,{139,227},15,16,17,19,20,20,21,21,23,28,30,26,23,20,18,17,15,15,13,13,13,13,12,13,13,13,14,15,16,18,20,22,22,21,20,20,22,23,24,23,20,17,15,15,14,14,13,13,13,14}, {blue block,{387,328},46,45,40,36,34,33,32,30,31,33,33,35,39,43,47,51,50,49,49,50,50,50,48,46,46,45,41,37,33,32,29,28,28,28,28,29,30,32,36,40,47,50,49,49,50,52,54,53,50,48}, {red block,{345,197},40,41,42,42,43,43,41,41,40,37,32,30,28,26,25,24,24,24,25,27,28,30,34,37,39,39,39,41,41,40,38,38,37,37,34,31,28,27,25,24,24,24,24,25,27,28,31,36,41,41}]",
    "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Given the following few token representations of a visual frame in the following format 'frame t-0: [{{object_id, {{centroid_x, centroid_y}},num_radial_points,radial_points}}]'..., predict the next frame in the same format'

### Input:
frame t-2: [{gripper,{172,132},78,70,73,81,86,93,105,158,187,186,202,141,156,156,116,26,22,18,16,16,14,14,14,14,15,16,16,179,120,109,167,195,173,158,147,140,136,134,134,123,113,103,95,84,77,74,72,73,77,77}, {table,{320,318},320,323,331,345,335,274,236,209,191,178,170,164,162,162,164,170,178,191,209,236,274,335,346,332,324,321,324,332,346,331,312,298,291,133,130,237,219,208,201,197,197,199,205,217,231,254,285,332,331,323}, {yellow block,{191,315},26,28,28,32,34,35,36,37,38,39,43,41,40,39,37,36,36,36,36,35,34,32,32,31,32,33,32,29,28,27,26,27,27,27,29,30,32,35,41,47,49,48,42,37,3

frame t+1: [{gripper,{168,107},43,44,46,47,83,88,110,102,111,125,132,73,71,84,101,99,89,79,81,81,72,71,66,36,36,38,40,42,46,58,63,70,104,128,120,114,110,109,109,110,91,64,59,54,50,46,44,42,43,43}, {table,{316,317},324,327,335,349,337,276,237,211,192,180,172,165,163,163,165,171,180,192,211,237,276,337,341,328,320,317,320,328,341,327,308,295,140,157,240,236,219,208,201,198,197,199,207,217,232,254,286,332,335,327}, {yellow block,{170,222},26,28,29,33,35,35,36,38,40,40,40,38,37,37,36,36,37,38,36,35,34,32,33,33,33,33,31,27,26,27,26,27,27,28,29,30,34,39,46,46,46,38,34,32,29,28,27,25,25,26}, {green block,{214,74},15,16,16,17,17,18,20,21,23,24,26,27,27,27,24,22,19,15,13,13,12,11,0,0,0,0,0,0,0,0,0,0,30,35,30,27,24,22,20,18,17,17,17,16,16,15,15,14,15,16}, {blue block,{387,328},46,45,40,36,34,33,31,30,31,31,33,35,39,43,47,51,50,49,49,49,50,50,48,47,46,45,41,37,33,32,30,28,29,28,28,29,30,32,36,40,47,50,49,50,50,52,54,53,49,48}, {red block,{345,197},40,41,42,42,43,43,41,41,40,37,32,30,28,27,25,24,2

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [13]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [14]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\n{\n  "id": "1470719797,\n  "name": "Eiffing, or, the, or, the, or, the, the, the, the, the, the, the, the, the, the, the, the, the,']

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [15]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [16]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [17]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>